
Identify the Goal, Metrics(KPI), Audience,

1. Problem Statement
1. Stakeholders

Parameters, Descriptions, Criteria

What visualizations do I need? How should information be presented?


Data: https://www.kaggle.com/datasets/jmullan/agricultural-land-values-19972017?select=Combined+Data+-+Pasture.csv

See if you can find the same granularity data for crop yields so that more comparison can be made and a yield/value metric can be extracted.

Problem Statement:
Does higher land value mean higher yield?  What are the trends for the two key metrics over time? Which regions, landuses, and states have the highest values, yields and values/yields. Show land value and crop yield at starting year, middle year and last available year on a map visual.

Agri Value by year
Agri Yield by year


^ Is this scope creep? It doesn't feel like just using the value would be enough for a compelling SQL DB structure, PBI dashboard or data story compared to the interplay between the value of the land, yield, and type of crop.


1. Set up SQL DB **
1. Load unclean land value data and use Python to clean data **
1. Load unclean crop production data and use Python to clean data **
1. Populate SQL DB with 2 tables, one for landval and one for cropprod **
1. Join tables into view and use to pull into Dashboard **
1. Use data to create visuals that visualize the relationship between the data and answer the questions in the problem statements.

## SQL DB Creation

## Land Value

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from sqlalchemy import *

# Constant of Region Names
REGIONS= ("Northeast", "Lake", "Corn Belt", "Northern Plains", "Appalachian", "Southeast", "Delta", "Southern Plains", "Mountain", "Pacific")

In [3]:
# Pull in cropland data 
cropland_df = pd.read_csv('data\Combined Data - Cropland.csv', header = 4)
# Remove junk columns
cropland_df.drop(columns = ['2','h','2016-2017'],index=[0,1],inplace=True)
# Remove random characters
cropland_df.replace('[1234567890]/', '', regex=True,inplace=True)
# Drop NA values and reset index
cropland_df.dropna(inplace=True)
cropland_df.reset_index(drop=True,inplace=True)

# Pull in farm real estate data 
farmre_df = pd.read_csv('data\Combined Data - Farm Real Estate.csv',header=4,encoding_errors = 'replace')
# Remove junk columns
farmre_df.drop(columns = ['1','h','2016-2017'],index=[0,1],inplace=True)
# Remove random characters
farmre_df.replace('[1234567890]/', '', regex=True,inplace=True)
# Drop NA values and reset index
farmre_df.dropna(inplace=True)
farmre_df.reset_index(drop=True,inplace=True)

# Pull in pasture data 
pasture_df = pd.read_csv('data\Combined Data - Pasture.csv',header=4)
# Remove junk columns
pasture_df.drop(columns = ['5','h','2016-2017'],index=[0,1],inplace=True)
# Remove random characters
pasture_df.replace('[1234567890]/', '', regex=True,inplace=True)
# Drop NA values and reset index
pasture_df.dropna(inplace=True)
pasture_df.reset_index(drop=True,inplace=True)




In [4]:
# Get the rows for each region
cropland_reg = cropland_df[cropland_df['Region and state'].isin(REGIONS)]
farmre_reg = farmre_df[farmre_df['Region and state'].isin(REGIONS)]
pasture_reg = pasture_df[pasture_df['Region and State'].isin(REGIONS)]

# Make a list of the indices of each region's row
cropland_list = list(cropland_reg.index)
farmre_list = list(farmre_reg.index)
pasture_list = list(pasture_reg.index)

# Append the last row of each dataframe
cropland_list.append(cropland_df.shape[0])
farmre_list.append(farmre_df.shape[0])
pasture_list.append(pasture_df.shape[0])

# Instantiate index dictionaries for each dataframe
cropland_idx_dict = {}
farmre_idx_dict = {}
pasture_idx_dict = {}

# For the each element of cropland_list, create a new entry in the index dictionary with the first and last index of that region's states (i.e. 'Northeast': [1, 7])
for i in range(0,len(cropland_list)-1):
  cropland_idx_dict[REGIONS[i]] = [cropland_list[i]+1,cropland_list[i+1]]

# For the each element of farmre_list, create a new entry in the index dictionary with the first and last index of that region's states (i.e. 'Northeast': [1, 7])
for i in range(0,len(farmre_list)-1):
  farmre_idx_dict[REGIONS[i]] = [farmre_list[i]+1,farmre_list[i+1]]

# For the each element of pasture_list, create a new entry in the index dictionary with the first and last index of that region's states (i.e. 'Northeast': [1, 7])
for i in range(0,len(pasture_list)-1):
  pasture_idx_dict[REGIONS[i]] = [pasture_list[i]+1,pasture_list[i+1]]

# Rename 'Region and state' column to 'State' and create new 'Region' Column
cropland_df.rename(columns={'Region and state':'State'},inplace=True)
cropland_df['Region'] = ''
# for each region, assign the region's name to 'Region' Column
for i in cropland_idx_dict.keys():
  cropland_df.loc[cropland_idx_dict[i][0]:cropland_idx_dict[i][1], 'Region'] = i
# Drop all rows that contain the regional data
cropland_df.drop(index=cropland_list[:-1],inplace=True)
# Drop row that contains cumulative US data and reset index
cropland_df.drop(index=[53],inplace=True)
cropland_df.reset_index(drop=True,inplace=True)

# Melt dataframe so each row only has one year and one acre value, convert acre value to numeric, and drop all NAs
cropland = pd.melt(cropland_df,id_vars=['State','Region'],var_name='Year',value_name='Acre Value')
cropland['Acre Value'] = pd.to_numeric(cropland['Acre Value'],errors='coerce')
cropland.dropna(inplace=True)
# Assign Land use as Cropland and reset index
cropland['Land Use'] = 'Cropland'
cropland.reset_index(drop=True,inplace=True)

# Rename 'Region and state' column to 'State' and create new 'Region' Column
farmre_df.rename(columns={'Region and state':'State'},inplace=True)
farmre_df['Region'] = ''
# for each region, assign the region's name to 'Region' Column
for i in farmre_idx_dict.keys():
  farmre_df.loc[farmre_idx_dict[i][0]:farmre_idx_dict[i][1], 'Region'] = i
# Drop all rows that contain the regional data
farmre_df.drop(index=farmre_list[:-1],inplace=True)
# Drop row that contains cumulative US data and reset index
farmre_df.drop(index=[58],inplace=True)
farmre_df.reset_index(drop=True,inplace=True)

# Melt dataframe so each row only has one year and one acre value, convert acre value to numeric, and drop all NAs
farmre = pd.melt(farmre_df,id_vars=['State','Region'],var_name='Year',value_name='Acre Value')
farmre['Acre Value'] = pd.to_numeric(farmre['Acre Value'],errors='coerce')
farmre.dropna(inplace=True)
# Assign Land use as Cropland and reset index
farmre['Land Use'] = 'Farm Real Estate'
farmre.reset_index(drop=True,inplace=True)

# Rename 'Region and state' column to 'State' and create new 'Region' Column
pasture_df.rename(columns={'Region and State':'State'},inplace=True)
pasture_df['Region'] = ''
# for each region, assign the region's name to 'Region' Column
for i in pasture_idx_dict.keys():
  pasture_df.loc[pasture_idx_dict[i][0]:pasture_idx_dict[i][1], 'Region'] = i
# Drop all rows that contain the regional data
pasture_df.drop(index=pasture_list[:-1],inplace=True)
# Drop row that contains cumulative US data and reset index
pasture_df.drop(index=[52],inplace=True)
pasture_df.reset_index(drop=True,inplace=True)

# Melt dataframe so each row only has one year and one acre value, convert acre value to numeric, and drop all NAs
pasture = pd.melt(pasture_df,id_vars=['State','Region'],var_name='Year',value_name='Acre Value')
pasture['Acre Value'] = pd.to_numeric(pasture['Acre Value'],errors='coerce')
pasture.dropna(inplace=True)
# Assign Land use as Cropland and reset index
pasture['Land Use'] = 'Pasture'
pasture.reset_index(drop=True,inplace=True)

# Concatenate all dataframes into final table
df = pd.concat([cropland,farmre,pasture],ignore_index=True)

# Initalize SQL Connection
metadata = MetaData()
engine = create_engine("mssql+pyodbc://ADHARSHVANCHI-P/Agricultural_Land_Data?driver=ODBC+Driver+17+for+SQL+Server")

# # Load final table into SQL as agricultural_landvalue
# df.to_sql('agricultural_landvalue',con=engine,if_exists='replace',index=False)

# # Update table by removing leading and trailing whitespaces
# with engine.connect() as connection:
#     connection.execute(text("UPDATE agricultural_landvalue SET state = TRIM(state)"))
#     connection.commit()

In [5]:
df

,State,Region,Year,Acre Value,Land Use
0,Delaware,Northeast,1997,2500.0,Cropland
1,Maryland,Northeast,1997,3050.0,Cropland
2,New Jersey,Northeast,1997,8500.0,Cropland
3,New York,Northeast,1997,1020.0,Cropland
4,Pennsylvania,Northeast,1997,2650.0,Cropland
...,...,...,...,...,...
2760,Utah,Mountain,2017,1050.0,Pasture
2761,Wyoming,Mountain,2017,510.0,Pasture
2762,California,Pacific,2017,2700.0,Pasture
2763,Oregon,Pacific,2017,700.0,Pasture


## Crop Production

In [47]:
cropprod_df = pd.read_csv('data\Crop Production in $.csv')
cropprod_df.head()
cropprod_df = cropprod_df[['Year','State','Value']]
cropprod_df.dropna(inplace=True)
cropprod_df.reset_index(drop=True,inplace=True)
cropprod_df.rename(columns={'Value':'Crop Yield'},inplace=True)
cropprod_df['State'] = cropprod_df['State'].apply(str.title)
cropprod_df.to_sql('crop_production',con=engine,if_exists='replace',index=False)

,State,variable,value
0,Alabama,Year,2.017000e+03
1,Arizona,Year,2.017000e+03
2,Arkansas,Year,2.017000e+03
3,California,Year,2.017000e+03
4,Connecticut,Year,2.017000e+03
...,...,...,...
1051,Virginia,Crop Yield,9.509280e+08
1052,Washington,Crop Yield,5.418545e+09
1053,West Virginia,Crop Yield,1.111640e+08
1054,Wisconsin,Crop Yield,3.483346e+09
